# Week 5: Fast pipelines, profiling

## Automatic mixed precision

In [ ]:
!nvidia-smi

In [27]:
import gc
from time import time

import torch
import torch.nn as nn
from tqdm.auto import trange

In [8]:
# Timing utilities
start_time = None
def start_timer():
    global start_time
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    start_time = time()

def end_timer_and_print(local_msg):
    torch.cuda.synchronize()
    end_time = time()
    print("\n" + local_msg)
    print("Total execution time = {:.3f} sec".format(end_time - start_time))
    print("Max memory used by tensors = {} bytes".format(torch.cuda.max_memory_allocated()))

In [9]:
def make_model(in_size, out_size, num_layers):
    layers = []
    for _ in range(num_layers - 1):
        layers.append(torch.nn.Linear(in_size, in_size))
        layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(in_size, out_size))
    return torch.nn.Sequential(*tuple(layers)).cuda()

In [39]:
batch_size = 512 # Try, for example, 128, 256, 513.
in_size = 8192
out_size = 8192
num_layers = 3
num_batches = 50
epochs = 3

# Creates data in default precision.
# The same data is used for both default and mixed precision trials below.
# You don't need to manually change inputs' dtype when enabling mixed precision.
data = [torch.randn(batch_size, in_size, device="cuda") for _ in range(num_batches)]
targets = [torch.randn(batch_size, out_size, device="cuda") for _ in range(num_batches)]

loss_fn = torch.nn.MSELoss().cuda()

In [42]:
net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)

start_timer()
for epoch in trange(epochs):
    for input, target in zip(data, targets):
        output = net(input)
        loss = loss_fn(output, target)
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Default precision:")

  0%|          | 0/3 [00:00<?, ?it/s]


Default precision:
Total execution time = 42.162 sec
Max memory used by tensors = 6375868928 bytes


In [43]:
net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)

start_timer()
for epoch in trange(epochs):
    for input, target in zip(data, targets):
        net.half()
        output = net(input.half())
        loss = loss_fn(output, target.half())
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Half precision:")

  0%|          | 0/3 [00:00<?, ?it/s]


Half precision:
Total execution time = 61.051 sec
Max memory used by tensors = 6979913728 bytes


TBD: half precision + loss scaling

In [24]:
net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

start_timer()

for epoch in trange(epochs):
    for input, target in zip(data, targets):
        # Runs the forward pass under autocast.
        with torch.cuda.amp.autocast():
            output = net(input)
            # output is float16 because linear layers autocast to float16.
            assert output.dtype is torch.float16

            loss = loss_fn(output, target)
            # loss is float32 because mse_loss layers autocast to float32.
            assert loss.dtype is torch.float32

        # Exits autocast before backward().
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("AMP:")


AMP:
Total execution time = 126.234 sec
Max memory used by tensors = 6308579328 bytes


In [25]:
# Constructs scaler once, at the beginning of the convergence run, using default args.
# If your network fails to converge with default GradScaler args, please file an issue.
# The same GradScaler instance should be used for the entire convergence run.
# If you perform multiple convergence runs in the same script, each run should use
# a dedicated fresh GradScaler instance.  GradScaler instances are lightweight.
scaler = torch.cuda.amp.GradScaler()
net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

start_timer()

for epoch in trange(epochs):
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast():
            output = net(input)
            loss = loss_fn(output, target)

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(opt)

        # Updates the scale for next iteration.
        scaler.update()

        opt.zero_grad() # set_to_none=True here can modestly improve performance

end_timer_and_print("AMP with loss scaling:")


AMP with loss scaling:
Total execution time = 130.169 sec
Max memory used by tensors = 6308579328 bytes


In [26]:
use_amp = True

net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

start_timer()
for epoch in trange(epochs):
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast(enabled=use_amp):
            output = net(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Mixed precision:")


Mixed precision:
Total execution time = 130.090 sec
Max memory used by tensors = 6308579328 bytes


### Gradients modification

In [ ]:
for epoch in range(epochs):
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast():
            output = net(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()

        # Unscales the gradients of optimizer's assigned params in-place
        scaler.unscale_(opt)

        # Since the gradients of optimizer's assigned params are now unscaled, clips as usual.
        # You may use the same value for max_norm here as you would without gradient scaling.
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.1)

        scaler.step(opt)
        scaler.update()
        opt.zero_grad() # set_to_none=True here can modestly improve performance

## Batching

Standard batching approach is just to stack tensors aquired with `__getitem__`.

- Sequences with varying length
- Different labels dimensions

In [57]:
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

brain: pad everything to a fixed max_length

big brain: pad only in the collate_fn

ultra duper big brain: presort data to sample sequences smartly, preserving +- the same length in a batch

In [67]:
# TODO: substitute with DataSet with gititem
lines = [
    "One thing I don't know why",
    "It doesn't even matter how hard you try",
    "Keep that in mind, I designed this rhyme",
    "To explain in due time",
    "All I know",
    "Time is a valuable thing",
    "Watch it fly by as the pendulum swings",
    "Watch it count down to the end of the day",
    "The clock ticks life away",
    "It's so unreal",
    "Didn't look out below",
    "Watch the time go right out the window",
    "Tryin' to hold on, did-didn't even know",
    "I wasted it all just to watch you go",
    "I kept everything inside and even though I tried",
    "It all fell apart",
    "What it meant to me will eventually",
    "Be a memory of a time when I tried so hard",
    "I tried so hard and got so far",
    "But in the end it doesn't even matter",
    "I had to fall to lose it all",
    "But in the end it doesn't even matter"
]
labels = torch.randint(2, (len(lines), ))
dataset = list(zip(lines, labels))
tokenizer = get_tokenizer("basic_english")


def yield_tokens(data_iter):
    for text, label in data_iter:
        yield tokenizer(text)

        
vocab = build_vocab_from_iterator(yield_tokens(iter(dataset)), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))


def collate_batch(batch: list[tuple[str, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
    text_list, label_list = [], []
    for _text, _label in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
  
    text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=0)
    label_list = torch.tensor(label_list, dtype=torch.int64)
  
    return text_list, label_list


dataloader = DataLoader(
    dataset, 
    batch_size=2, 
    collate_fn=collate_batch,
    shuffle=True
)

for x, _ in dataloader:
    print(f"Current batch:\n{x}\n")

Current batch:
tensor([[ 1,  3, 71, 10, 79,  0,  0,  0,  0,  0,  0],
        [78,  3,  5, 52, 66, 20, 38,  3,  6,  7, 17]])

Current batch:
tensor([[39,  3,  6, 60, 25, 32,  0,  0],
        [67, 26,  2, 40,  3,  6, 17, 84]])

Current batch:
tensor([[82,  1, 63,  5, 62, 85, 43,  0,  0],
        [ 2, 58, 44, 54, 21,  7, 75,  2, 19]])

Current batch:
tensor([[ 4, 34, 76, 59, 30,  0,  0,  0,  0],
        [ 2, 81,  1,  8, 56,  5, 12, 27, 23]])

Current batch:
tensor([[22,  9,  4, 15,  1, 14,  3,  6,  7, 18],
        [ 1, 14,  3,  6,  7, 18, 53, 16, 27, 77]])

Current batch:
tensor([[57, 73,  9, 65, 20,  2, 37, 74, 69],
        [ 8,  2, 17,  0,  0,  0,  0,  0,  0]])

Current batch:
tensor([[11, 55, 13, 80, 26,  0,  0,  0],
        [12,  1, 49, 33, 29,  4, 68, 72]])

Current batch:
tensor([[12,  4, 11, 23, 70, 25,  4, 86],
        [ 2, 51,  5, 46,  5, 61,  1,  8]])

Current batch:
tensor([[ 1,  8, 48, 28,  0,  0,  0,  0,  0,  0],
        [22,  9,  4, 15,  1, 14,  3,  6,  7, 18]])

Current bat

Also check out `transformers.DataCollatorWithPadding` at https://huggingface.co/docs/transformers/main_classes/data_collator

## Data preprocessing with DALI
https://docs.nvidia.com/deeplearning/dali/user-guide/docs/index.html

In [ ]:
#!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda102

Augmentations: https://docs.nvidia.com/deeplearning/dali/user-guide/docs/examples/image_processing/augmentation_gallery.html

TODO: select simple example 

For images check out source for ResNet50 on ImageNet: https://docs.nvidia.com/deeplearning/dali/user-guide/docs/examples/use_cases/pytorch/resnet50/pytorch-resnet50.ht

## Albumentations
TBD

## Streaming datasets

What if we do not want to wait until dataset is downloaded? Terrabytes of data.

What if we do not have enough disk space?

https://huggingface.co/docs/datasets/dataset_streaming.html

In [72]:
# !pip install datasets
from datasets import load_dataset

In [73]:
dataset = load_dataset('oscar', "unshuffled_deduplicated_en", split='train', streaming=True)
print(next(iter(dataset)))

{'id': 0, 'text': 'Mtendere Village was inspired by the vision of Chief Napoleon Dzombe, which he shared with John Blanchard during his first visit to Malawi. Chief Napoleon conveyed the desperate need for a program to intervene and care for the orphans and vulnerable children (OVC) in Malawi, and John committed to help.\nEstablished in honor of John & Lindy’s son, Christopher Blanchard, this particular program is very dear to the Blanchard family. Dana Blanchard, or Mama Dana as she is more commonly referred to at Mtendere, lived on site during the initial development, and she returns each summer to spend the season with her Malawian family. The heart of the program is to be His hands and feet by caring for the children at Mtendere, and meeting their spiritual, physical, academic, and emotional needs.\nMtendere Village is home to 134 children, living in 16 homes with a housemother and several brothers and sisters. This family environment is one that many of the children have never pre

In [74]:
# what to do with thing we are used to? shuffling?
shuffled_dataset = dataset.shuffle(buffer_size=10_000, seed=42)

In [75]:
next(iter(shuffled_dataset))

{'id': 892,
 'text': 'In this role, she oversees the day-to-day operations of the agency’s motoring services divisions (Vehicle Titles & Registration, Motor Vehicles, Motor Carrier, Enforcement, Consumer Relations and the Automobile Burglary & Theft Prevention Authority) to ensure they are constantly improving and identifying opportunities to become more efficient and effective in service delivery.\nMellott came to the TxDMV from Alaska’s Division of Motor Vehicles where she most recently served as deputy executive director and acting executive director where she led a major initiative to modernize and improve the customer service experience. Previous positions at the Alaska DMV include oversight of all large field offices and leading the driver licensing program.\nMellott serves on the American Association of Motor Vehicle Administrators Unconventional Vehicle Working Group and has worked collaboratively with representatives from across the country to develop best practices for states

In [76]:
print(dataset.n_shards)

670


In [77]:
shuffled_dataset.set_epoch(epoch) # seed -> seed + epoch